<a href="https://colab.research.google.com/github/nullhypothesiss/Sign-Language-Recognition/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install split-folders

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import os
import splitfolders
import numpy as np

In [2]:
import zipfile
zip_ref = zipfile.ZipFile('sl.zip', 'r')
zip_ref.extractall('/content')
#zip_ref.close()

In [3]:
print(zip_ref)

<zipfile.ZipFile filename='sl.zip' mode='r'>


In [4]:
train_df = pd.read_csv("/content/sign_mnist_train.csv")
test_df = pd.read_csv("/content/sign_mnist_test.csv")

In [5]:
def print_data_info(directory):
  if os.path.exists(directory):
    print(f'The current structure of {directory} is:')
    for i,folder in enumerate(os.listdir(directory)):
      print(f'{i+1}. {folder} folder contains:')
      folder_path=os.path.join(directory,folder)
      folder_imgs=0
      for cls in os.listdir(folder_path):
        cls_imgs=len(os.listdir(os.path.join(folder_path,cls)))
        print('- {} {} image'.format(cls_imgs, cls))
        folder_imgs+=cls_imgs
    print(f'* Total Images: {folder_imgs} (%{folder_imgs/3000.0*100}) *\n')

print_data_info('/content/sl')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
datagen = ImageDataGenerator()
train_ds = datagen.flow_from_directory("/content/sign_mnist_train")
print("classes: ", train_ds.class_indices)
imgs, labels = train_ds.next()

Found 0 images belonging to 0 classes.
classes:  {}


In [8]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

In [9]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

In [10]:
x_train = train_df.values
x_test = test_df.values

In [11]:
x_train = x_train / 255
x_test = x_test / 255

In [12]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [13]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(x_train)

In [14]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))

In [15]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 75)        300       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 75)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 50)        2

In [16]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 128) ,epochs = 10 , validation_data = (x_test, y_test))

model.save('smnist.h5')

Epoch 1/10
215/215 [==============================] - 124s 568ms/step - loss: 1.0366 - accuracy: 0.6762 - val_loss: 3.9737 - val_accuracy: 0.1546
Epoch 2/10
215/215 [==============================] - 123s 571ms/step - loss: 0.2176 - accuracy: 0.9297 - val_loss: 1.7218 - val_accuracy: 0.4951
Epoch 3/10
215/215 [==============================] - 119s 552ms/step - loss: 0.1053 - accuracy: 0.9661 - val_loss: 0.1145 - val_accuracy: 0.9675
Epoch 4/10
215/215 [==============================] - 123s 572ms/step - loss: 0.0651 - accuracy: 0.9789 - val_loss: 0.0253 - val_accuracy: 0.9941
Epoch 5/10
215/215 [==============================] - 116s 540ms/step - loss: 0.0489 - accuracy: 0.9849 - val_loss: 0.0554 - val_accuracy: 0.9794
Epoch 6/10
215/215 [==============================] - 110s 512ms/step - loss: 0.0380 - accuracy: 0.9875 - val_loss: 0.0516 - val_accuracy: 0.9823
Epoch 7/10
215/215 [==============================] - 115s 534ms/step - loss: 0.0308 - accuracy: 0.9901 - val_loss: 0.1560 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
 !pip install -U scikit-learn

In [18]:
!pip install evaluate

In [26]:
import evaluate

In [25]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]*100)

Test loss: 0.11384225636720657
Test accuracy: 96.16564512252808
